<a href="https://colab.research.google.com/github/matthiaskozubal/Chat_with_data/blob/main/code/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat with data
Dynamic, LLM-powered quering over PDFs, using either predefined PDFs or yours (from your google drive).

## How to use
1. **Run code**:
  - Click `Runtime` -> `Run all` or hit `Ctrl+F9`
2. **Choose PDFs source**: When prompted in the 'User Input' section:
  - type **author's** or hit **Enter** (to use predefined PDFs)
  - type **mine** and ensure (to use your PDFs):
    - a folder named `PDFs` is in your Google Drive with your PDFs inside

# Setup

## Libraries

In [12]:
from google.colab import drive

## Constants

In [31]:
PREDEFINED_FOLDER_PATH = 'content/drive/My Drive/PDFs'
AUTHORS_GOOGLE_DRIVE_SHARED_FOLDER_LINK = 'pass'

## Functions

In [29]:
def get_user_choice(verbose=False):
  PDFs_source_choice = input(f"Choose PDF source (author's/user's) [default: author's]: ")
  if PDFs_source_choice not in ['author\'s', 'user\'s']:
    PDFs_source_choice = 'author\'s'

  if verbose:
    print(PDFs_source_choice)

  return PDFs_source_choice

# PDFs_source_choice = get_user_choice(verbose=True)

Choose PDF source (author's/user's) [default: author's]: 
author's


In [22]:
def connect_to_google_drive():
	drive.mount('/content/drive')

In [34]:
def query_over_PDFs(PDFs_source_choice, PREDEFINED_FOLDER_PATH, verbose=False):

  if PDFs_source_choice == 'author\'s':
    folder_link = AUTHORS_GOOGLE_DRIVE_SHARED_FOLDER_LINK
    print(f"...")
  elif PDFs_source_choice == 'user\'s':
    connect_to_drive()
    print(f"...")
    if os.path.exists(PREDEFINED_FOLDER_PATH):
        pdf_files = [f for f in os.listdir(PREDEFINED_FOLDER_PATH) if f.endswith('.pdf')]

  # querying over PDFs
  #...

  if verbose:
    print(f"{folder_link}: {PREDEFINED_FOLDER_PATH}")


query_over_PDFs(PDFs_source_choice, PREDEFINED_FOLDER_PATH, verbose=True)

pass: content/drive/My Drive/PDFs


In [35]:
def get_user_question(verbose=False):
  user_question = input(f"Ask a question to the PDFs")
  if user_question == '':
    user_question = '''
      What is the bias and variance in terms of machine learning?
      '''
  if verbose:
    print(user_question)

  return user_question

# user_question = get_user_question(verbose=True)

# Main

In [36]:
PDFs_source_choice = get_user_choice(verbose=True)

Choose PDF source (author's/user's) [default: author's]: 
author's


In [37]:
user_question = get_user_question(verbose=True)

Ask a question to the PDFs

      What is the bias and variance in terms of machine learning.
      
